# Testing the `channel` function given in the pdf

In [133]:
import numpy as np

In [232]:
def channel(chanInput):
    chanInput = np.clip(chanInput,-1,1)
    erasedIndex = np.random.randint(3) 
    chanInput[erasedIndex:len(chanInput):3] = 0
    return chanInput + np.sqrt(10)*np.random.randn(len(chanInput))

In [40]:
channelInput = np.array([1,-1,1,1,1,-1])
channelOutput = channel(channelInput)

print(channelOutput)

[ 7.09291894 -2.59928026 -1.48739072  2.76715194 -0.53892032 -2.80916199]


# Command to get the real channel output

In [ ]:
python3 client.py --input_file=in.txt --output_file=out.txt --srv_hostname iscsrv72.epfl.ch --srv_port 80

In [13]:
np.savetxt("in.txt", channelInput)

In [15]:
channelOutput = np.loadtxt("out.txt")
print(channelOutput)

[ 8.86527571  2.56791895 -5.904792   -2.13201648  8.97192555 -0.96314209]


# Functions to get a string to channel input format ([-1,1,1,-1,...]) and back

In [89]:
def strToBits(string):
    res = []
    byte_string = string.encode('utf-8')
    for b in byte_string:
        bit_array = bin(b)[2:]
        bit_array = '00000000'[len(bit_array):] + bit_array
        res.extend(bit_array)
    return res

def stringToChannelInput(string):
    bits = np.array(strToBits(string), dtype='int64')
    return 2*bits - 1

In [90]:
channelOutput = stringToChannelInput("abcd")
print(channelOutput)
print(len(channelOutput))

[-1  1  1 -1 -1 -1 -1  1 -1  1  1 -1 -1 -1  1 -1 -1  1  1 -1 -1 -1  1  1
 -1  1  1 -1 -1  1 -1 -1]
32


In [93]:
def channelOutputToString(channelOutput):
    bits = ((channelOutput+1)/2).astype('int64').tolist()
    byte_string = ""
    for char_index in range(len(bits)//8):
        bit_list = bits[char_index*8:(char_index+1)*8]
        byte = chr(int(''.join([str(bit) for bit in bit_list]), 2))
        byte_string += byte
    return byte_string

    

In [94]:
channelOutputToString(channelOutput)

'abcd'

# Encoding

In the homework, they talk about "4 equally likely messages". Hence I propose to encode every pair in the message into a triple. This effecitvely makes the messages 1.5 longer. 

* For messages that are of size $2k$:
    We simply do the pair to triple transformation as stated above. We obtain a signal of size $3k$

* For messages that are of size $2k-1$:
    We do the same transform as above for the first $2(k-1)$. Then we double the last character. We therefore obtain a message of size $3(k-1)+2=3k-1$. On the decoder, we can infer $J$ and still recover the last character no matter what. We also know that the input message in of odd size as the decoder will receive a message of size not divisible by $3$.

In [204]:
codewords = [[1, 1, 1], [1, -1, -1], [-1, 1, -1], [-1, -1, 1]]
tupleToCodewordDict = {(1, 1): codewords[0], (1, -1): codewords[1], (-1, 1): codewords[2], (-1, -1): codewords[3]}

# The channel only accepts -1 and 1
def transformTuple(subarray):
    return tupleToCodewordDict[tuple(subarray)]

def augmentSignal(array, K):
    n = array.shape[0]
    newArray = np.array([], dtype='int64')
    for i in range(n//2):
        newArray = np.append(newArray, transformTuple(array[2*i:2*i+2]) * K)

    if n % 2 != 0:
        newArray = np.append(newArray, [array[n-1], array[n-1]] * K)

    return newArray

# array is an np.array with values -1 and 1.
# The output is an np.array with values -1 and 1 (channel only accepts those)
def encode(array, K=10):
    return augmentSignal(array, K)

We will employ the straightforward decoding specified in the last subquestion of the homework. In particular, we first guess $J$ and then $H$.

The homework only considered the case where we had one triple.
If the encoded signal is of odd size, we ignore the last two characters. Hence we work with a number of samples divisible by $3$ always (say $3k$). We have:
$$J_{MAP}((y_1, \dotsc, y_k)) = \operatorname{argmin}_{j=1}^{3} \Pi_{l=1}^{k} (y_l)_j$$
In particular, we have a way less error prone $J$ estimator than with a single triple.

In [240]:
def jmap(array):
    p1 = p2 = p3 = 1
    # by '//3' we ignore the last 2 samples if they're here.
    for i in range(len(array)//3):
        p1 *= array[3*i]**2
        p2 *= array[3*i+1]**2
        p3 *= array[3*i+2]**2
    if p1 < p2 and p1 < p3:
        return 0
    if p2 < p1 and p2 < p3:
        return 1
    return 2

def cutJ(array, j, K):
    n = array.shape[0]
    remaining = (n%3)*K
    cutoff = n - remaining

    beforeCutoffIndex = [i%3!=j for i in range(cutoff)]
    afterCutoffIndex = [(i+cutoff)%3!=j for i in range(remaining)]

    left = array[:cutoff]
    right = array[cutoff:]

    return left[beforeCutoffIndex], right[afterCutoffIndex]

def adjacentIndex(j):
    if j == 0:
        return 1, 2
    if j == 1:
        return 0, 2
    else:
        return 0, 1

def transformKPairs(subarray, j, K, debug):
    l, r = adjacentIndex(j)
    
    if debug:
        print("Subarray:")
        print(subarray)

    minIndex = -1
    minSum = sys.maxsize
    for i in range(4):
        s0 = 0
        for j in range(K):
            s0 += (subarray[2*j] - codewords[i][l])**2 + (subarray[2*j+1] - codewords[i][r])**2
        if s0 < minSum:
            minSum = s0
            minIndex = i

    if debug:
        print("Min index:")
        print(minIndex)

    if debug:
        print("Result:")
        print(codewords[minIndex])

    return codewords[minIndex][:2]

def reduceSignal(leftReducedArray, rightReducedArray, j, K, verboseDebug):
    n = leftReducedArray.shape[0]
    newArray = np.array([], dtype='int64')

    for i in range(n//(2*K)):
        newArray = np.append(newArray, transformKPairs(leftReducedArray[2*K*i:2*K*(i+1)], j, K, verboseDebug))

    if rightReducedArray.shape[0] != 0:
        codewords_simple = [-1, 1]

        minIndex = -1
        minSum = sys.maxsize
        for i in range(2):
            s0 = 0
            for e in rightReducedArray:
                s0 += (e - codewords_simple[i])**2
            if s0 < minSum:
                minSum = s0
                minIndex = i
                
        if verboseDebug:
            print("Min index:")
            print(minIndex)
            print("Codeword:")
            print(codewords_simple[minIndex])

        newArray = np.append(newArray, codewords_simple[minIndex])

    return newArray 

# array is an np.array with values -1 and 1.
def decode(array, K=10, debug=False, verbose=False):
    n = array.shape[0]
    j = jmap(array)
    if debug:
        print(f"j: {j}")
    leftReducedArray, rightReducedArray = cutJ(array, j, K)
    if debug:
        print(f"after cutJ: {leftReducedArray}")
        print(f"after cutJ: {rightReducedArray}")

    reducedArray = reduceSignal(leftReducedArray, rightReducedArray, j, K, debug and verbose)

    return reducedArray

In [237]:
# K has to be coprime with 3.
K = 97

# x = np.array([1,1,-1,1,-1,-1, 1])
x = np.array([-1, -1, 1, -1,  1, -1,  1,  1, -1,  1, -1, -1, -1,  1,  1,  1, -1])
# x = np.array([1,1,1,-1,-1,1,-1,-1,1,1])
print("Original:")
print(x)
x = encode(x, K)
print("Encoded:")
print(x)
x = channel(x)
print("After channel:")
print(x)
x = decode(x, K, debug=False)
print("After decoding:")
print(x)

Original:
[-1 -1  1 -1  1 -1  1  1 -1  1 -1 -1 -1  1  1  1 -1]
Encoded:
[-1 -1  1 ... -1 -1 -1]
After channel:
[-0.74220403 -5.22803974  2.49917131 ...  4.37517291 -2.57627309
 -5.09180845]
After decoding:
[ 1 -1  1 -1 -1 -1 -1  1  1  1  1 -1  1  1  1  1 -1]
/Users/xavier/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in double_scalars
  import sys
/Users/xavier/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in double_scalars
  
/Users/xavier/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in double_scalars
  """


## Testing the accuracy of the above encoding

In [241]:
import random

errs = 0
trials = 100 
maxSize = 100
choices = [-1, 1]
K = 97

for i in range(trials):
    n = random.randint(0, maxSize)
    x = np.array([random.choice(choices) for i in range(n)], dtype='int64')

    y = encode(x, K)
    y = channel(y)
    y = decode(y, K, debug=False)

    if not np.array_equal(x, y):
        errs+=1
        # print("==== ERROR ====")
        # print(f"x: {x}")
        # print(f"y: {y}")

print(f"Accuracy: {(float(trials)-float(errs))/float(trials)}")

/Users/xavier/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in double_scalars
  import sys
/Users/xavier/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in double_scalars
  
/Users/xavier/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in double_scalars
  """
Accuracy: 0.41
